In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from DrawtexDataset import DrawtexDataset
from DrawtexModel import DrawTexModel

#### Hyperparameters

In [2]:
TRAIN_BATCH_SZ = 128
TEST_BATCH_SZ = 1000
EPOCHS = 1
LEARN_RATE = 0.01
CLASS_CNT = 78

device: str = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


#### Dataloader setup

In [3]:
data_set = DrawtexDataset(transforms.ToTensor())
TRAIN_SIZE = int(0.8 * len(data_set))
TEST_SIZE = len(data_set) - TRAIN_SIZE
train_set, test_set = torch.utils.data.random_split(data_set, [TRAIN_SIZE, TEST_SIZE])

train_loader = DataLoader(
    dataset=train_set,
    batch_size=TRAIN_BATCH_SZ,
    shuffle=True,
    num_workers=4
)

test_loader = DataLoader(
    dataset=test_set,
    batch_size=TEST_BATCH_SZ,
    shuffle=False,
    num_workers=4
)

model = DrawTexModel().to(device)
print(model)

DrawTexModel(
  (relu): ReLU()
  (conv1): Conv2d(1, 150, kernel_size=(9, 9), stride=(1, 1), bias=False)
  (conv1_bn): BatchNorm2d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(150, 200, kernel_size=(9, 9), stride=(1, 1), bias=False)
  (conv2_bn): BatchNorm2d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(200, 300, kernel_size=(9, 9), stride=(1, 1), bias=False)
  (conv3_bn): BatchNorm2d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(300, 500, kernel_size=(9, 9), stride=(1, 1), bias=False)
  (conv4_bn): BatchNorm2d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(500, 800, kernel_size=(9, 9), stride=(1, 1), bias=False)
  (conv5_bn): BatchNorm2d(800, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lin1): Linear(in_features=20000, out_features=78, bias=False)
  (lin1_bn): BatchNorm1d(78, eps=1e-05, momentum=0.1, af

In [4]:
tens = torch.rand((2, 1, 45, 45)).to(device)

output1 = model(tens)

print(output1)

tensor([[-1.0000,  1.0000,  0.9998, -0.8419, -0.9999, -0.9998, -0.9998,  0.9999,
         -1.0000,  0.9999,  1.0000,  0.9990, -1.0000,  0.9017,  0.9996, -0.9999,
         -0.9999,  0.9998,  0.9998, -1.0000,  0.9991,  0.9999,  1.0000, -0.9999,
          0.9999,  0.9999, -0.9999,  0.9998,  1.0000,  0.9938,  0.9998,  0.9999,
         -1.0000,  0.9988,  0.9997, -0.9954, -0.9998, -0.9995,  0.9999, -0.9995,
         -0.9995,  0.9998,  0.9999, -1.0000,  0.9999,  0.9995, -1.0000, -0.9999,
          0.9998, -0.9973,  0.9999,  0.9999, -1.0000,  0.9989,  1.0000, -1.0000,
          0.9951, -1.0000,  0.9986,  0.9999, -0.9999, -0.9999,  1.0000,  1.0000,
         -0.9977,  0.9992, -1.0000, -0.9984,  1.0000,  0.9839,  0.9999, -0.9998,
          0.9999,  1.0000, -0.9990,  1.0000, -0.9994, -0.9999],
        [ 1.0000, -1.0000, -0.9998,  0.8419,  0.9999,  0.9998,  0.9998, -0.9999,
          1.0000, -0.9999, -1.0000, -0.9990,  1.0000, -0.9017, -0.9996,  0.9999,
          0.9999, -0.9998, -0.9998,  1.0000, 

In [5]:

def train():
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=LEARN_RATE)
    steps = len(train_loader)
    for epoch in range(EPOCHS):
        for i, (img, label) in enumerate(train_loader):
            img: torch.Tensor = img.to(device, non_blocking=True)
            label: torch.Tensor = label.to(device, non_blocking=True)
            output = model(img).to(device)
            loss: torch.Tensor = criterion(output, label).to(device)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if i % 10 == 0:
                print(f"Epoch {epoch + 1}/{EPOCHS}, Batch {i + 1}/{steps}, Loss {loss.item():.4f}")
    torch.save(model.state_dict(), "./DrawTexModel.pth")
    torch.save(optimizer.state_dict(), "./Optimizer.pth")

train()


Epoch 1/1, Batch 1/2324, Loss 4.8679
Epoch 1/1, Batch 11/2324, Loss 2.9351
Epoch 1/1, Batch 21/2324, Loss 2.7233
Epoch 1/1, Batch 31/2324, Loss 2.2919
Epoch 1/1, Batch 41/2324, Loss 2.2890
Epoch 1/1, Batch 51/2324, Loss 2.1070
Epoch 1/1, Batch 61/2324, Loss 2.0257
Epoch 1/1, Batch 71/2324, Loss 1.9609
Epoch 1/1, Batch 81/2324, Loss 1.7730
Epoch 1/1, Batch 91/2324, Loss 1.9109
Epoch 1/1, Batch 101/2324, Loss 1.4832
Epoch 1/1, Batch 111/2324, Loss 1.4727
Epoch 1/1, Batch 121/2324, Loss 1.5127
Epoch 1/1, Batch 131/2324, Loss 1.5213
Epoch 1/1, Batch 141/2324, Loss 1.4244
Epoch 1/1, Batch 151/2324, Loss 1.4434
Epoch 1/1, Batch 161/2324, Loss 1.3736
Epoch 1/1, Batch 171/2324, Loss 1.3135
Epoch 1/1, Batch 181/2324, Loss 1.3706
Epoch 1/1, Batch 191/2324, Loss 1.3479
Epoch 1/1, Batch 201/2324, Loss 1.1961
Epoch 1/1, Batch 211/2324, Loss 1.2068
Epoch 1/1, Batch 221/2324, Loss 1.1593
Epoch 1/1, Batch 231/2324, Loss 1.2232
Epoch 1/1, Batch 241/2324, Loss 1.2027
Epoch 1/1, Batch 251/2324, Loss 1.28

In [6]:
with torch.no_grad():
    correct = 0
    total = 0
    for img, labels in test_loader:
        img = img.to(device, non_blocking= True)
        labels = labels.to(device, non_blocking= True)

        output = model(img)
        _, prediction = torch.max(output, 1)
        total += labels.size(0)
        correct += (prediction == labels).sum().item()

    acc = 100.0 * correct / total
    print(f"Accuracy: {acc}%")

Accuracy: 96.64465632942886%
